<img src ="https://velog.velcdn.com/images/sjinu/post/ebe451cc-ee94-459c-b07f-df22d823ac0b/1231233123.PNG">

### DETR 논문
https://arxiv.org/pdf/2005.12872.pdf

### Code
https://github.com/facebookresearch/detr

object detection 방법들에 쓰이는 것처럼 non-maximum suppression이나 anchor generation 등과 같이 손수 디자인해야하는 요소들을 효과적으로 제거함으로써, detection 파이프라인을 간소화하였다. 우리가 DEtection TRansformer(DETR)이라고 부르기로 한 새로운 프레임워크의 주요 구성 요소는 transformer의 encoder-decoder 구조와 '양자간 매칭(bipartite matching)'을 통해 유니크한 예측을 하게끔 하는 set-based global loss라 할 수 있다. 학습된 object queries의 fixed small set만 주어지면, DETR은 object와 global image context 사이의 관계를 추론하고, 최종적인 예측 set을 곧바로 반환한다(in parallel with transformer). 새로운 모델은 개념적으로 굉장히 간단하고, specilaized library를 필요로 하지 않는다(이전 모델과의 차이점). DETR은 COCO object detetion dataset을 기반으로, 아주 잘 정의됐고, 최적화가 잘 된 Faster R-CNN과 동급의 정확도 및 런타임 속도를 보였다. 또한, DETR은 손쉽게 일반화할 수 있어 통합된 방식으로 panoptic segmentation을 생성할 수도 있다

object detection의 최종 목표는 관심 있는 각 object에 대해 category label과 bounding boxes 집합을 예측하는 것이다. 현재 많은 detector들은 이러한 set prediction problem을 간접적으로 다룬다. 예를 들면, surrogate regression과 classification problem을 수 많은 proprosal을 통해(Faster R-CNN(2015), Cascade R-CNN(2019)) anchors를 통해(Focal loss for dense object detection(2017)), 또는 window center를 통해(Objects as points(2019), Fully convolutional one-stage object detection(2019))정의한다.

위 모델들의 성능은 거의 겹치는 예측들을 후처리하거나, anchor set을 디자인하거나, 휴리스틱하게 target boxes를 anchor에 할당하는 데(Bridging the gap between anchor-based and anchor-free detection via adaptive training sample selection(2019))에 크게 의존한다.

아무튼, 우리는 위와 같은 과정을 간소화 하기 위해서 suurogate task를 패스하고 direct set prediction을 수행하는 방법론을 제안한다. 이 end-to-end 철학은 기계 번역이나 대화 인식과 같은 굉장히 복잡하게 구조화된 예측 태스크에서의 진보를 이끌었지만, 아직 object detection에는 없었다. 기존의 시도들은 사전 지식의 다른 형태를 추가하거나(End-to-end people detection in crowded scenes, Learning non-maximum suppression 등), 경쟁력 있는 성능을 보여주지 못했다. 본 연구에서는 이 갭을 줄이고자 한다.

## 구조 개요

우리는 학습 과정을 'direct set prediction problem'으로 다룬다.

첫번째로, sequence prediction에 쓰이는 유명한 모델 중 하나인 transformer의 encoder-decoder 구조를 채택하였다. transform의 self-attention mechanism(sequence 내 elements들 사이의 모든 pairwise 상호작용을 모델링하는 매커니즘)은 이 구조가 removing duplicate prediction과 같은 set prediction의 제약을 특히 다루기 쉽게 만들어 주었다.

두번째로, 우리의 DETR 모델은 한 번에 모든 OBJECT를 예측하기 위해, 예측 object와 ground-truth object 사이의 양자간 매칭(bipartite matching) 을 수행하는 a set loss function(여러 개을 통해 end-to-end로 학습된다. 특히, spatial anchor(?)나 non-maximal suppression과 같은 사전지식을 인코딩 해야하는 여러 수작업 요소들을 버림으로써 detection pipeline을 간소하게 했다.

## 기존 모델 vs DETR 특징

기존에 존재하는 대부분의 detection method들과는 다르게, DETR은 cutomized layer를 필요로 하지 않으며, 따라서 기본적인 CNN이나 transformer classes를 포함한 어떠한 framework에서도 재생상될 수 있다.

direct set prediction과 관련해 이전의 연구들과 비교했을 때, DETR의 주된 특징은 1. 양자간 매칭 loss와 2. (non-autoregressive) parallel decoding을 포함한 transformer(BERT(2019), Parallel wavenet(2017), Mask-predict(2019) 등 기계번역 연구들)의 결합이라고 볼 수 있을 것 같다.

기존의 (visual 관련 분야의) direct set prediction 연구들은 RNN을 통해 autoregressive decoding에 중점을 두었다. 우리 모델의 matching loss function은 ground truth object의 '유니크한' 예측을 할당하므로 예측된 object들의 permuation(순서, 순열)에는 변하지 않기 때문에, 병렬적으로 autoregressive decoding task를 제거할 수 있다.

성능평가 또한 진행했는데, Fast R-CNN은 계속해서 많은 수작업을 겪었기 때문에, 논문 발간 당시보다 성능이 굉장히 좋아진 상태였음에도 불구하고, 우리의 DETR 모델은 그와 비슷한 성능을 보였다(on COCO dataset). 더 정확히는, DETR은 (아마 트랜스포머의 non-local computations(?) 때문에) large object에 대한 성능이 좋았다. 하지만 small objects에 대한 성능은 그리 좋지 않았다. 우리는 이런 문제가 Faster R-CNN에 적용된 FPN의 개발과 같은 방식으로 개선될 수 있을 거라 믿는다.

Training Settings은 기존의 object detector들과는 여러 관점에서 다르다. 새로운 모델은 추가적으로 긴 학습 스케줄을 필요로 하며, transformer에 있는 보조적인 decoding loss 또한 사용해야 한다.

DETR의 디자인 기풍은 쉽게 더욱 복잡한 태스크로도 확장될 수 있다. 본 연구에서는 pre-trained DETR의 top을 기반으로 학습된 simple segmentation head가 Panoptic Segmentation(최근에 유명세를 얻은 어려운 pixel-level recognition task)에서 경쟁력 있는 모델을 성능 상 앞질렀다는 것을 보여줄 것이다.

## Related work

1. bipartite matching losses for set prediction
2. encoder-decoder based on the transformer
3. parrallel decoding
4. object detection methods

## DETR Model

detection 태스크에서 direct set predictions을 위해선 두 가지가 필요하다.

1. ground truth boxes와 predicted boxes 간에 'unique matching'을 할당하게끔 하는 set prediction loss

2. a set of objects들을 예측하고, 이들의 관계를 모델링하는 architecture.

※ 2.에서, 예측은 당연히 single pass로 행해져야 합니다. architecture와 관련해서는 3.2에서 딥하게 다룹니다.

## Object detection set prediction loss

DETR은 decoder를 통해서 단 한번의 pass로 고정된 개수인 N개의 예측을 반환합니다. 이 때, N은 image 내 전형적인 object의 개수보다 훨씬 커야한다. 학습에서의 주된 어려움은, ground truth 와 관련해 predicted objects에 점수를 부여하는 데 있다.(예를 들어, class, position, size)

우리의 loss는 multiple-object 관점에서 ground-truth와 prediction 간에 이상적인 'bipartite matching' 을 생성하고, 그 후 object 단위에서의 loss를 최적화합니다.

첫번째 과정으로, y를 set of objects의 ground truth라 하고,  
y^​ ={y^​i​ }i=1N​ 을 set of N predictions이라 하자. 보통 N을 넉넉하게 설정하기 때문에, y는 ϕ(no object)로 패딩된 N size 집합이라고 볼 수 있다. 그러면, N개의 ground truth 집합과 N개의 prediction 집합 사이의 'bipartite matching'을 찾기 위해서, 낮은 cost를 갖는 N개의 요소에 대한 permutation을 탐색한다.

즉, Lmatch​ (yi​ ,y^​σ(i)​ )를 ground truth yi​ 와 index σ(i)를 갖는 prediction  
y^​σ(i)​ 사이의 pair-wise matching cost라 할 때, 아래와 같은 permuation을 찾을 수 있게 된다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2Fbb09f209-1ddf-4e3b-ad66-5295a68542e9%2Fimage.png">

※ 최적의 assignment는 Hungarian algorithm을 사용해 효율적으로 계산된다("End-to-end people detection in corwded scence. In: CVPR(2015)").

matching cost는 class prediction과 predicted <-> ground truth boxes 간 similarity 두 가지 모두를 고려한다. ground truth set의 각 요소 i 관점에서 살펴보자. 이 때, c 
i
​
 를 target class label 이라하고, b 
i
​
 ∈[0,1] 
4
 를 ground truth box의 중심좌표 와 높이, 너비 에 해당하는 좌표라 할 때, y 
i
​
 =(c 
i
​
 ,b 
i
​
 )처럼 나타낼 수 있다.

※ 위의 중심 좌표와 높이, 너비는 image size 대비 크기이기 때문에 0과 1사이로 나타낼 수 있다.

또한, 인덱스 σ(i)에 대한 예측을 위해 class c 
i
​
 의 예측확률을  
p^​σ(i)​ (ci​ )라 정의하고, predicted box를  
b^σ(i)​ 로 정의한다. 위와 같이 노테이션을 줬을 때, matching cost L 
match
​
 (y 
i
​
 , 
y
^
​
  
σ(i)
​
 )를 아래와 같은 식으로 정의할 수 있다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F39fa4b90-7935-400a-9101-1c2eb0ab47e3%2Fimage.png">

이렇게 matching을 찾는 과정은 사실 modern detector인 proposal(RPN)이나 anchors(FPN)를 ground truth objects에 매칭하는 heuristic assignment rules과 같은 역할을 한다. 다른 점은, duplicates가 없는 direct set prediction을 위한 1-1 매칭을 찾아야 한다는 것이다.

두번째 과정은 이전 스텝에서 매칭한 모든 pairs에 대한 Hungarian loss를 연산하는 것이다. 우리는 loss를 흔한 object detector의 loss들과 유사하게 정의한다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F6c2e37b3-d49c-4672-9d26-7a9e86804996%2Fimage.png">

※ 즉, class prediction과 box의 loss를 위해 negative log-likelihood의 linear combination을 사용합니다.

이 때,  
σ
^
 은 식(1) 에서 계산한 optimal assignment이다. 실질적으로, class imbalance를 설명하기 위해 c 
i
​
 =ϕ일 때의 log-probability term을 factor 10정도로 down-wieght한다.

※ 이에 대해서는 연구 "This is analogous to how Faster R-CNN training procedure balances positive/negative proposals by subsampling"에서 Faster R-CNN의 학습 과정이 positive/negative proposals의 밸런스를 맞추는 방법(서브샘플링)과 유사합니다.

또한, OBJECT와 ϕ 사이의 cost는 preidction에 의존하지 않는다. 즉, 이 상황에서 cost는 상수이다. 그 상황에선 log-probabilities 대신  
p^​σ^ (i)​ (ci​ )를 사용한다. 이는 class prediction term을 L 
box
​
 (∗,∗)와 상응하게 만들어 준다(성능향상).

### Bounding box loss

위의 matching cost 와 식 (2) 에서 두번째 term에 해당하는 loss이다.
modern detector는 최초의 예측에 대한 그래디언트로서 box prediction을 수행하지만, 우리는 directly하게 box prediction을 수행한다. 이는 실행을 간단하게 만들긴 하지만, loss의 relative scaling issue를 불러일으킨다.

흔히 쓰이는 ℓ 
1
​
  loss는 relative error가 비슷하더라도 small box와 large box간 다른 scale을 갖는다. 이를 해결하기 위해(즉, scale-invariant를 보장하기 위해) linear combination of the ℓ 
1
​
  loss and generalized IoU loss L 
iou
​
 (∗,∗)를 사용한다.

일반적으로, λ 
iou
​
 ,λ 
L1
​
 ∈R를 hyperparameter로 설정할 때, 우리의 box loss L 
box
​
 (b 
i
​
 , 
b
^
  
σ(i)
​
 )를 아래와 같은 식으로 정의한다.

<img src ="https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F9b06c8d5-c2c0-48b6-94ab-4f4f671f3dce%2Fimage.png">

※ 역시, 이 두가지 loss는 batch 내부의 object 개수로 normalize합니다.

## DETR architecture

<img src ="https://velog.velcdn.com/images%2Fsjinu%2Fpost%2Fb2bbfd51-30f6-4992-b68b-47b1b0200e8b%2Fimage.png">

위 DETR 구조의 메인요소는 아래와 같이 3개로 이루어져 있다.

1. compact feature representation을 추출하는 CNN backbone
2. encoder-decoder transformer
3. 최종적인 detection 예측을 반환하는 simple feed forward network (FFN)

많은 현대 object detector들과는 달리, DETR은 CNN backbone과 transformer 구조를 제공하는 대부분의 딥러닝기반 frame work에 (몇백줄의 코드만으로) 적용될 수 있다(심지어 inference code는 50줄 미만이다).

### Backbone

input image를 x 
img
​
 ∈R 
3×H 
0
​
 ×W 
0
​
 
 라 할 때, 전통적인 CNN backbone 모델은 낮은 차원의 activation map f∈R 
C×H×W
 를 생성한다. 우리는 주로 C=2048,H= 
32
H 
0
​
 
​
 ,W= 
32
W 
0
​
 
​
 의 값을 사용한다.

※ 이 때, input images들은 batch 단위로 묶여 같은 차원 (여기선 H 
0
​
 ,W 
0
​
 ) 을 갖도록 적절히 0-padding이 실행됩니다. 당연히 batch 내 가장 큰 image의 차원을 따릅니다.

### Transformer encoder

우선, 1×1의 convolution이 high-level activation map f 의 차원을 C에서 d로 낮춘다. 즉, 새로운 feature map은 z 
0
​
 ∈R 
d×H×W
 가 된다. encoder는 input으로 sequence를 받기 때문에, z 
0
​
 라는 공간차원(spatial dimensions)을 one dimension으로 낮추어야 한다. 즉, d×HW의 feature map이 된다.

각 encoder layer는 standard 구조를 가지며, multi-head self-attention module과 feed forward network (FFN) 으로 이루어져 있다. transformer 구조는 순서에 무관하기 때문에(permutation-invariant) 우리는 이 구조에 fixed positional encodings를 추가했고, 이는 각 attention layer의 input에 더해진다.

※ 위의 fixed positional encoding 개념은 "Image transformer. In: ICML (2018)"와 "Attention augmented convolutional networks. In: ICCV (2019)"를 기반으로 합니다.

※ 또한, "Attention is all you need. In: NeurIPS (2017)"에 쓰인 구조 중 하나를 따릅니다.

### Transformer decoder

decoder 역시 transformer의 standard 구조를 갖는다. 즉, multi-headed self attention, encoder decoder attention 매커니즘을 사용해 N embeddings of size d 를 transform한다.

※ 여기서 N은 충분히 큰 object(bboxes)의 개수, d는 feature map의 spatial dimension에 해당합니다.

기존의 transformer와 다른 점은, 우리 모델은 각 decoder layer에서 병렬적으로 N object를 디코딩한다는 것이다.

※ 'Attention is all you need'에서는 '한 번에 한 번씩' output sequence를 예측하는 autoregressive model을 사용합니다. 

decoder 역시 순서에 무관하기 때문에, 다른 결과들을 생성하기 위해선 N input embedding 또한 달라야 한다. 이 input embeddings는 positional encodings를 학습하는데, 우리는 이를 object queries라 부른다. encoder에서와 유사하게, 이 object queries를 각 attention layer의 input에 더한다. N object queries는 decoder에 의해 output embedding으로 변환된다.
그 후, FFN에 의해 이 output embedding은 box coordinates(좌표)와 class labels로 '독립적으로' 디코딩 된다. 그 결과 N개의 final predictions이 나온다.
모델은 이런 임베딩들에 대해 self attention과 encoder-decoder attention을 사용하면서, 모든 objects에 대해, 그들 간 pair-wise relations(개별 조합들을 테스트)를 사용하여, global하게 추론한다. 그 과정에서 모델은 전체 이미지를 context로 사용할 수 있다.

### Prediction feed-forward networks (FFNs)

최종적인 예측은 ReLU activation layer, hidden dimension d layer, linear projection layer로 이루어진 3-layer perceptron에 의해 연산된다. FFN은 input image에 대한 normalized center coordinates, height, width를 예측하며, linear layer는 softmax function을 이용해 class label을 예측한다.

※ 위에서 말했다 시피, N개의 bounding boxes를 예측하는 것은 개수를 고정했기 때문에 image 내 존재하는 실제 object 개수들보다 (보통) 훨씬 커야하며, 다른 detector들과 유사하게 object가 존재하지 않는다는 것(즉, background)을 표현하기 위해 추가적인 class label ϕ를 사용합니다.

※ linear layer랑 FFN이 배타적으로 구분되는 layer인지? Appendix를 읽을 필요가 있을 것 같다.

### Auxiliary(보조) decoding losses
학습 도중에 decoder auxiliary losses를 사용하는 것이 (특히, 모델이 각 class의 object들의 올바른 개수를 반환하는 데) 굉장히 도움되었다.
매 decoder layer뒤에 prediction FFNs과 Hungarian loss를 추가하였으며, 모든 predictions FFNs은 그 parameter를 공유하게끔 설정하였다.
또한, different decoder layer에 있는 prediction FFN에 input을 정규화 하기 위해 추가적인 shared layer-norm 을 사용한다.

## Technical details

해당 저자들은 아래와 같은 Setting으로 학습을 진행한다.

### DETR

- transformer : AdamW with learning rate 10 
−4
 
- backbone : AdamW with learning rate 10 
−5
 
- weight decay : 10 
−4

모든 트랜스포머의 가중치는 Xavier init을 통해 초기화되며, backbone 모델은 ImageNet에 pretrained된 ResNet을 batchnorm layers frozen 시킨 채로 사용한다.
저자는 두 개의 backbone 모델에 대한 결과(ResNet-50, Resnet-101)를 보여주며, 각각 DETR과 DETR-R101으로 이름 짓는다.

연구 Li, Y. et al, Fully convolutional instance-aware semantic
segmentation. In: CVPR (2017) = https://arxiv.org/abs/1611.07709
에 나와있는 것처럼, backbone의 last stage에 dilation을 더하고(stage : BottleNeck으로 보면 될 듯), 해당 state 내 first convolution의 stride를 제거함으로써 feature resolution을 증가시켰다.

### ※ "Fully convolutional instance-aware semantic segmentation"

In the original ResNet, the effective feature stride (the
decrease in feature map resolution) at the top of the network is 32. This is too coarse for instance-aware semantic
segmentation. To reduce the feature stride and maintain the
field of view, the “hole algorithm” [3, 29] (Algorithme a`
trous [30]) is applied. The stride in the first block of conv5
convolutional layers is decreased from 2 to 1. The effective
feature stride is thus reduced to 16. To maintain the field
of view, the “hole algorithm” is applied on all the convolutional layers of conv5 by setting the dilation as 2.

해당 논문은 원래 semantic segmentation을 위한 논문이다. 하지만, instance-awre semantic segmentation을 하기에는 Resnet의 기존의 feature stride인 32는 너무 coarse해, 마지막 stage의 첫번째 convolution layer의 stride를 2에서 1로 줄이는(즉, stride를 없애는) 과정을 거쳤고, hole algorithm을 마지막 state의 모든 convlution layer에 적용함으로써 field of view를 효과적으로 유지하였다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2Ffa1bc081-c9ef-46de-8cc8-4ec2708d04f0%2Fimage.png">

아무튼, 위와 같은 과정을 거친경우 DETR-DC5, DETR-DC5-R101(dilated C5 stage)라 부른다. 이러한 수정은 resolution을 2배 상승시켰기 때문에 작은 object를 detect하는 성능을 개선하였으나, self-attention 과정에서 연산량이 16배 상승하기 때문에 전반적인 연산량 또한 2배 상승하였다.

※ Feature maps size 2배 올리는 데 연산량 2배면 꽤나 합리적인 trade-off인 듯 하다.

또한 저자들은 이미지의 가로, 세로 중 짧은 길이를 480~800으로, 긴 길이가 최대 1333이 되게끔 scale augmentation을 사용한다. encoder의 self-attention 과정을 통해 global relations ships를 학습할 수 있게끔, 학습 도중에 random crop augmentations을 진행하고, 이로 인해 약 1 AP 정도의 성능 향상을 이루어 냈다.

※ 구체적으로는, 학습 이미지가 50%의 확률로 RANDOM RECTANGULAR PATCH를 갖게끔 crop되고, 이는 다시 800-1333 사이로 resizing 됩니다.

dropout은 기본적으로 0.1을 사용했다(in transformer).

inference time에서는 당연히 몇 개의 slot은 empty class를 예측한다. 하지만, 성능을 높히기 위해 empty class를 second highest scroing class로 대체함으로써, 2 AP 정도의 성능 향상을 달성했다.

마찬가지로 학습 때 400 epochs이 지나면 학습률을 10배 감소시켰는데(총 500 epochs), 이로 인해 200 epochs이 지나고 학습률을 감소시킨 모델(총 300 epochs)보다 성능이 1.5 AP가량 증가했다.

추가적인 학습 hyperparameters는 section A.4에 존재합니다.

## Comparison with Faster R-CNN

### Number of encoder layers

아래 그림 (Figure 3)에서는, 학습 모델의 마지막 encoder layer의 attention maps을 시각화한다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F7a04c410-5655-4ab7-bc5f-915891242952%2Fimage.png">

encdoer는 instance를 개별로 분리할 수 있을 뿐더러, 이런 특징은 decoder가 object를 추출하고 localize하는 데 도움이 되는 것으로 보인다.

### Number of decoder layers

Figure 3의 encoder attention 시각화와 유사하게, 아래 그림(Figure 6)에서는 decoder attentions을 시각화할 수 있다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F98ca5b0b-b337-49d8-8488-7e02abecaa73%2Fimage.png">

decdoer의 attention은 주로 물체의 말단(머리, 다리 등)에 집중되는 것을 볼 수 있다. (하나의 가설로는) encoder가 global attention을 통해 instance를 분리한 다음이라면, decoder는 class와 object boundaries를 찾기 위해 object의 말단만을 봐도 된다는 것일 수 있다.

### Importance of FFN
### Importance of positional encodings

우리 모델에서는 아래와 같이 2개의 positional encodings이 쓰인다.

1. spatial positional encodings

2. output positional encodings(object queries)

positional encodings에 대해 다양한 조합을 실행해보았고, 결과는 아래 table 3과 같다.



<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F371a0fec-e7fc-413b-a253-e64faae54672%2Fimage.png">

위 테이블을 보면, Output positional encodings은 거의 무조건 필요하다는 것을 볼 수 있었다. 그래서 positional encodings을 decoder input에만 한 번만 투입하기도 하고, 모든 decoder attention layer에 더해보기도 하였다. spatial positional encodings을 아예 제외하고, output positional encdoings만 decoder intput에 통과시켰을 때에는 7.8 AP 정도 하락했다. "Attention is all you need" 연구에서 사용한 것처럼, fixed sine spatial positional encodins와 output encdoings을 input에만 적용했을 때에도 1.4AP 정도 하락했다. attention layer에 leanred spatial encodings를 적용했을 때도 큰 차이는 없었다. 특히, spatial positional encodings을 사용하지 않았을 때도 성능이 1.3 AP 정도밖에 떨어지지 않았다.

※ encodings를 attention layer들에 적용했을 때는 모든 layer에 대해 동등하게 적용되며, output encdoings(object queries)는 항상 학습됩니다.

이러한 ablations들을 고려했을 때, transformer components인 global self-attention in encoder, FFN, multiple decoder layers, positional encodings 등은 최종적인 object detection performance에 큰 기여를 했음을 알 수 있다.

## A. Appendix

### A.1 Preliminaries: Multi-head attention layers

이 단락에서는 attention mechanism의 일반형태에 대해서 알아보자. attention mechanism은 "Attention is all you need"연구를 따르며, 오직 positional encodings만 "On the relationship between self-attention and convolutional layers" 연구를 따른다.

### Multi-head
d차원의 M heads를 갖는 multi-head attention의 일반 형태는 아래와 같은 함수로 주어진다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F5efe07dd-9055-4675-970b-3bc4d046bafa%2Fimage.png">

※ d 
′
 는 d 
′
 =d/M으로 정의되며, 아래괄호는 행렬이나 텐서의 사이즈이다.

#### Input

X 
q
​
  : 길이 N 
q
​
 의 query sequence
 
X 
kv
​
  : 길이 N 
kv
​
 의 key-value sequence
 
T : 소위 query, key, 그리고 value embeddings을 연산하기 위한 weight tensor

L : projection matrix

#### Output

X~q
​
  : query sequence와 같은 길이의 output.

※ 참고로, multi-head self- attention (mh-s-attn)은 X 
q
​
 =X 
kv
​
 인 special case이다.
즉, mh_s_attn(X,T,L)=mh_attn(X,X,T,L)

multi-head attention은 간단히 말하면, M개의 single attention을 concat한 다음 L로 project한 것이라고 보면 된다. 일반적으로는 residual connetions, dropout, layer normalization을 사용한다. 달리 말하면,  
X
~
  
q
​
 를 mh−attn(X 
q
​
 ,X 
kv
​
 ,T,L)로, X 
q
′
​
 를 attention heads의 concat이라 정의하면, 아래와 같은 식을 얻는다.

<img src ="https://velog.velcdn.com/images%2Fsjinu%2Fpost%2Fbf80266c-28f3-4f15-8531-a4a2774d286c%2Fimage.png">

위 식 (5),(6)에 concat(;), projection(L), dropout(dropout), residual connections(X 
q
​
 +...), layer normalization(layernorm)에 대한 개념이 모두 들어 있다.

### Single head

가중치 tensor로 T 
′
 ∈R 
3×d 
′
 ×d
 를 갖는 attention head attn(X 
q
​
 ,X 
kv
​
 ,T 
′
 )는 additional positional encoding P 
q
​
 ∈R 
d×N 
q
​
 
 와 P 
kv
​
 ∈R 
d×N 
kv
​
 
 에 의존한다.

attention head는, 우선 query와 key positional encodings를 더한 후, query, key, and value embeddings를 연산한다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F8145775a-5c5a-4345-bf8c-569a8ad28d4d%2Fimage.png">

※ T 
′
 는 T 
1
′
​
 ,T 
2
′
​
 ,T 
3
′
​
 의 concat이다. T 
′
 는 단순히 표기 중복 방지를 위해서 붙힌 것으로 보인다.
또한, 위 식에서 T 
n
′
​
 ,(n∈1,2,3)은 T 
n
′
​
 ∈R 
d 
′
 ×d
 이다.
예를 들어, 위 임베딩의 1번째 term은 d 
′
 ×N 
q
​
 차원, 2-3번째 term은 d 
′
 ×N 
kv
​
 차원이다.

attention weight α 는 query와 key의 내적을 한 다음 softmax를 씌움으로써 계산한다. 즉, 이로 인해 query sequence의 각 요소는 key-value sequence의 모든 요소에 방문한다고 볼 수 있다(아래 식을 보자).

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2Fb53c5712-4bca-4f87-b4f3-130aa1e801b9%2Fimage.png">

※ Q는 d 
′
 ×N 
q
​
 차원, K는 d 
′
 ×N 
kv
​
 차원이다.

※ i는 query index이고, j는 key-value index 임을 주목하자. 반면, d 
′
 는 single head의 차원이다(d는 M multihead의 차원이고, d 
′
 =d/M이므로). 즉, 위 식에서는 정규화 목적으로 쓰인 듯하다.

positional encodings은 본문 전체에서 많이 언급했듯, 학습되거나 고정될 수 있다. 단, (query/key-value sequence가 주어졌을 때) 모든 attention layer가 같은 가중치를 공유한다(디테일은 encoder and decoder에 포함).

final output은 attention weight에 의해 가중된 values의 aggregation은 아래와 같다. 즉, i-th row는의 output 아래 식에 의해 주어진다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2F25debadd-84d9-4647-8d74-b9a036603b94%2Fimage.png">

### Feed-forward network(FFN) layers

original transformer는 multi-head attention 과 FFN layer를 번갈아 가며 사용한다. FFN layer는 효율적인 multi-layer 1x1 convolutions이며, 우리의 경우 Md input과 output channels을 가진다. 또한, 우리가 고려한 FFN은 2-layer의 1x1 convolutions(with ReLU)로 이루어져 있다.

추가로, residual connection/dropout/layernorm이 2-layers 뒤에 이어진다.

## A.2 Losses

## A.3 Detailed architecture

positional encodings를 모든 attention layer에 통과시킨, DETR에 쓰인 transformer 모델의 디테일은 아래 그림과 같이 주어진다.

<img src = "https://velog.velcdn.com/images%2Fsjinu%2Fpost%2Fe5e0d736-a773-49a0-919d-6cc1aef5e9cd%2Fimage.png">

다시 inference 과정을 간단히 요약해보자.

1. Input image는 CNN backbone을 거쳐 image features 형태로 변환된다.
2. Image features는 spatial positional encodings과 함께 encoder로 투입된다.
    - 단, spatial positional encodings는 image features에 더해져, 모든 multi-head self-attention layer의 queries와 keys에 더해진다.
3. decoder는 encoder에서 나온 queries, output positional encoding(object queries), encoder memory를 받아 최종적인 class labels와 bounding boxes을 예측한다.
    - 단, 예측은 multiple multi-head self-ateention과 decoder-encoder attention을 통해 이루어진다(첫번째 decoder layer의 첫번째 self-attention layer는 스킵될 수 있음).